<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/python_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data clean 

BathroomsFull
BathroomsHalf
BathroomsTotalDecimal
BathroomsTotalInteger
BedroomsTotal
BuildingAreaTotal
City
CloseDate
ClosePrice
CumulativeDaysOnMarket
DaysOnMarket
GarageSpaces
Latitude
ListingContractDate
ListingId
ListPrice
ListSource
Longitude
LotSizeAcres
MlsStatus
OwnerName
OwnerPhone
OwnerPhoneAlternative
ParcelNumber
ParcelNumber2
PoolYN
PostalCode
PreviousListPrice
PreviousStatus
PropertySubType
PropertyType
PublicRemarks
PurchaseContractDate
SchoolDistrict
StandardStatus
StatusChangeTimestamp
StreetDirPrefix
StreetDirSuffix
StreetName
StreetNumber
StreetNumberNumeric
StreetSuffix
SubdivisionName
USProperty_MUI
YearBuilt



In [ ]:
#import package
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys


# Data cleaning

In [ ]:
import pandas as pd
import censusgeocode as cg
import numpy as np
from rets import Session
def findifNull(lo_left, lo_right, la_left, la_right, research, zip):
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {})'.format(lo_left, lo_right, la_left, la_right, zip))
    count = 0
    lis = []
    di = {}
    df = pd.DataFrame()
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
            di = item
    for key in di:
        lis.append(key)
    if(count >= 5):
        df = df.T
        df.columns = lis
        return df
    else:
        lo_left -= 0.001
        lo_right += 0.001
        la_left -= 0.001
        la_right += 0.001
        return findifNull(lo_left, lo_right, la_left, la_right, research, zip)
def findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip, a, time1, time2):
    basic = house_leftSqt
    count = 0
    df = pd.DataFrame()
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (YearBuilt = {}+), (YearBuilt = {}-), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (LivingArea = {}+), (LivingArea = {}-), (PostalCode = {})'.format(time1, time2, house_leftLo, house_rightLo, house_leftLa, house_rightLa, house_leftSqt, house_rightSqt, zip))
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
    if(count >= 5):
        df = df.T
        df.columns = column
        return df
    else:
        house_leftLa -= 0.001
        house_rightLa += 0.001
        house_leftLo -= 0.001
        house_rightLo += 0.001

  #Living Area
        if(a == 0):
            if(basic <= 1100):
                house_leftSqt -= 100
                house_rightSqt += 100
            elif(basic <= 2200):
                house_leftSqt -= 200
                house_rightSqt += 200
            else:
                house_leftSqt -= 300
                house_rightSqt += 300

            a += 1
        return findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip, a, time1, time2) 

    
def findfirst(result):
    count = 0
    for item in result:
        count += 1
    if(count == 0):
        return False
    else:
        return True    
    
def insertId(pulled_value, mls):
    
    column = []
    for key in pulled_value:
        column.append(key)
    print("\nThis value exists in Website")
  # get whole row value
  #get each value
    zip = pulled_value.get('PostalCode')
    house_leftLa = float(pulled_value.get('Latitude'))
    house_rightLa = float(pulled_value.get('Latitude')) 
    house_leftLo = float(pulled_value.get('Longitude'))
    house_rightLo = float(pulled_value.get('Longitude'))
    house_leftSqt = float(pulled_value.get('LivingArea'))
    house_rightSqt = float(pulled_value.get('LivingArea'))
    time1 = int(pulled_value.get('YearBuilt'))
    time2 = int(pulled_value.get('YearBuilt'))
    if time1 < 1990:
        time1 = 0 
        time2 = 1989
    elif time1 >=1990 and time1 <2011:
        time1 = 1990
        time2 = 2010
    elif time1 >= 2011 and time1 < 2022:
        time1 = 2011
        time2 = 2021
    else:
        time1 = 2022
        time2 = 9999
    a = 0
    print(house_leftSqt)
    result1 = findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip, a, time1, time2)        
    result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
    price = []
    for item in result1['ClosePrice']:
        item = float(item)
        price.append(item)
    result1['ClosePrice'] = price
    mean1 = result1['ClosePrice'].mean()
    top1 = list(result1['ClosePrice'])
    top1.sort(reverse = True)
    print('The mean for {} is {}'.format(mls, mean1))
    print('The top 3 for {} are {}'.format(mls, top1))
    return result1.to_csv('{}.csv'.format(mls))
def findAdd(address, city, state, zip):
    
    result = cg.address(address, city = city, state= state,zipcode = zip)
    lo_left = result[0]['coordinates']['x']
    lo_right = result[0]['coordinates']['x']
    la_left = result[0]['coordinates']['y']
    la_right = result[0]['coordinates']['y']
    result = findifNull(lo_left, lo_right, la_left, la_right, rets_client, zip)
    result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
    price = []
    for item in result['ClosePrice']:
        item = float(item)
        price.append(item)
    result['ClosePrice'] = price
    mean = result['ClosePrice'].mean()
    top = list(result['ClosePrice'])
    top.sort(reverse = True)
    print('The mean for {}, {}, {} is {}'.format(address, state, zip, mean))
    print('The top 3 for {}, {}, {} are {}'.format(address, state, zip, top))
    return result.to_csv('{},{},{}.csv'.format(address, state, zip))